In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from pathlib import Path
import imageio
from tqdm import tqdm_notebook as tqdm

In [3]:
from config import tracker2, bokeh_viz

In [4]:
DRAW = False

In [5]:
def draw_detect(img, hands):
    c = [(0,255,255), (255,255,0)] * 10
    
    for j,h in enumerate(hands):
        crd = h['lm'].astype('int')
        box = h['bbox'].astype('int')
        dydx = h['dydx'].astype('int')
        center = h['lm'][2].astype('int')
        
        cv2.arrowedLine(img, tuple(center-dydx), tuple(center), (255,255,255), 5, tipLength=0.2)
        
        for i in range(0,4):
            beg = tuple(box[i])
            end = tuple(box[(i+1) % 4])
            cv2.line(img, tuple(box[i]), tuple(box[(i+1) % 4]), c[j], 3)
    return img

def get_pad(hands, img_shape):
    assert len(img_shape) == 2
    
    boxes = np.c_[[x['bbox'] for x in hands]].reshape(-1, 2)
    
    max_ = boxes.max(axis=0).astype('int')+1
    min_ = boxes.min(axis=0).astype('int')-1
    
    
    p_min = np.maximum(-min_, 0)
    p_max = np.maximum(max_ - img_shape[::-1], 0)
    
    new_boxes = [(x['bbox'] + p_min).astype('int') for x in hands]
    
    return ((p_min[1],p_max[1]), (p_min[0], p_max[0]), (0,0)), new_boxes

def crop_hands(frame, hands):
    if len(hands) == 0:
        return []
    pad, boxes = get_pad(hands, frame.shape[:2])
    padded = np.pad(frame, pad, mode='constant')
    
    res = []
    
    for b in boxes:
        slc0 = slice(b[0,1], b[1,1])
        slc1 = slice(b[0,0], b[2,0])
        res.append(cv2.resize(padded[slc0, slc1], (256,256)))
    return res

In [6]:
source = Path("~/data/unlabelled/1cut/").expanduser()
vids = sorted(source.glob("*"))[16:]

In [7]:
for vid in tqdm(vids, leave=False):
    cap = imageio.get_reader(vid)
    frames = [x for x in cap]
    cap.close()

    hands = [tracker2(x) for x in tqdm(frames, leave=True)]

    if DRAW:
        save_path = vid.as_posix().replace("1cut", "3boxed")

        writer = imageio.get_writer(save_path, fps=30)

        for frame, hand in zip(frames, hands):
            writer.append_data(draw_detect(frame, hand))

        writer.close()

    cropped = [crop_hands(f, h) for f,h in zip(frames, hands)]

    seq = [
        [[]], #hand 0
        [[]],  #hand 1 
    ]

    n_prev = 2
    for crp in cropped:
        if n_prev != len(crp):
            for s, h in zip(seq, crp):
                s.append([])
            n_prev = len(crp)

        for s,h in zip(seq, crp):
            s[-1].append(h)

    save_path = vid.as_posix().replace("1cut", "4cropped")

    for i, hand_seqs in enumerate(seq):
        for j, seq in enumerate(hand_seqs):
            if len(seq) > 30:
                save_cur = save_path.replace(".mp4", f"_hand_{i}_segm_{j}.mp4")
                writer = imageio.get_writer(save_cur, fps=30)
                for frame in seq:
                    writer.append_data(frame)
                writer.close()